<a href="https://colab.research.google.com/github/ms9306/vaccine-sentiment-analysis/blob/main/beautiful_soup_fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Description: this program scrapes FAKE and REAL news data from a website (Politifact.com)

In [2]:
# Importing the libraries
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import urllib.request
import time

In [4]:
# Creating list to store scraped data
authors = []
dates = []
statements = []
source = []
targets = []

In [51]:
# Create function to scrape the website 
def scrape_website(page_number):
  page_num = str(page_number)
  URL = 'https://www.politifact.com/factchecks/list/?page=&'+page_num
  webpage = requests.get(URL)
  soup = BeautifulSoup(webpage.text, 'html.parser')

  # Get location of information 
  statement_footer = soup.find_all('footer', attrs={'class': 'm-statement__footer'}) #Location of the author & date
  statement_quote = soup.find_all('div', attrs={'class': 'm-statement__quote'}) # Location of the statement on the page
  statement_meta = soup.find_all('div', attrs={'class': 'm-statement__meta'}) # Location of the source 
  target = soup.find_all('div', attrs={'class': 'm-statement__meter'}) # Location of the target (score-card graphic)

  # Loop through statement footer for each quotation
  for i in statement_footer:
    link_1 = i.text.strip()
    name_and_date = link_1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name + ' ' + last_name

    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month + ' ' + day + ' ' + year

    dates.append(date)
    authors.append(full_name)

    # Loop through the statement_quote 
    for i in statement_quote:
      link2 = i.find_all('a') #Uses the a-tag (which refers to hyperlinks in HTML)
      statement_text = link2[0].text.strip()
      statements.append(statement_text)
    
    # Loop through the meta
    for i in statement_meta:
      link3 = i.find_all('a')
      source_text = link3[0].text.strip()
      source.append(source_text)
    
    # Loop through the target
    for i in target:
      link4 = i.find('div', attrs={'class': 'c-image'}).find('img').get('alt')
      targets.append(link4)

In [6]:
# Loop through 'n-1' webpages to scrape the data 
n = 2
for i in range(1, n):
  scrape_website(i)

In [63]:
# Create dataframe
data = pd.DataFrame(columns= ['author', 'statement', 'source', 'date', 'target'])

data['author'] = pd.Series(authors)
data['statement'] = pd.Series(statements)
data['source'] = pd.Series(source)
data['date'] = pd.Series(dates)
data['target'] = pd.Series(targets)

# Show the data
data

,author,statement,source,date,target
0,MiriamValverde,"""I don't think anybody is questioning the legi...",Kevin McCarthy,"May14,2021",pants-fire
1,PaulSpecht,Says “75% of third-graders in this year’s clas...,Pat McCrory,"May14,2021",half-true
2,CiaraO'Rourke,Says Benjamin Netanyahu gave a speech to Hamas...,Facebook posts,"May14,2021",false
3,LouisJacobson,“The research showed … that confidence overall...,Jeffrey Zients,"May14,2021",half-true
4,BrandonMulder,The voting rights bill “would register million...,Ted Cruz,"May14,2021",pants-fire
5,D.L.Davis,On supporting Liz Cheney for House Republican ...,Mike Gallagher,"May14,2021",full-flop
6,SashaHupka,"“After California taxes, Trey Lance's $34.1M r...",Instagram posts,"May13,2021",half-true
7,CiaraO'Rourke,Video shows woman filling plastic bags with ga...,Viral image,"May13,2021",false
8,AmySherman,“Watching the TV footage of those who entered ...,Andrew Clyde,"May13,2021",pants-fire
9,CiaraO'Rourke,Photos show gas costs nearly $10 in the Caroli...,Viral image,"May13,2021",false


In [60]:
# Function that gets a binary NUMBER from the target
def get_binary_num_target(text):
  if text == 'true':
    return 1
  else: 
    return 0

In [61]:
def get_binary_target(text):
  if text == 'true':
    return 'REAL'
  else:
    return 'FAKE'

In [65]:
# Create two new columns on the dataframe to include the above information
data['Binary_Target'] = data['target'].apply(get_binary_target)
data['Binary_Num_Target'] = data['target'].apply(get_binary_num_target)

In [66]:
# Show the updated dataframe
data

,author,statement,source,date,target,Binary_Target,Binary_Num_Target
0,MiriamValverde,"""I don't think anybody is questioning the legi...",Kevin McCarthy,"May14,2021",pants-fire,FAKE,0
1,PaulSpecht,Says “75% of third-graders in this year’s clas...,Pat McCrory,"May14,2021",half-true,FAKE,0
2,CiaraO'Rourke,Says Benjamin Netanyahu gave a speech to Hamas...,Facebook posts,"May14,2021",false,FAKE,0
3,LouisJacobson,“The research showed … that confidence overall...,Jeffrey Zients,"May14,2021",half-true,FAKE,0
4,BrandonMulder,The voting rights bill “would register million...,Ted Cruz,"May14,2021",pants-fire,FAKE,0
5,D.L.Davis,On supporting Liz Cheney for House Republican ...,Mike Gallagher,"May14,2021",full-flop,FAKE,0
6,SashaHupka,"“After California taxes, Trey Lance's $34.1M r...",Instagram posts,"May13,2021",half-true,FAKE,0
7,CiaraO'Rourke,Video shows woman filling plastic bags with ga...,Viral image,"May13,2021",false,FAKE,0
8,AmySherman,“Watching the TV footage of those who entered ...,Andrew Clyde,"May13,2021",pants-fire,FAKE,0
9,CiaraO'Rourke,Photos show gas costs nearly $10 in the Caroli...,Viral image,"May13,2021",false,FAKE,0
